In [1]:
# Import required libraries
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from torch.nn.functional import softmax
import os
from pathlib import Path

c:\Users\Fawaz\Desktop\AI_Text_Detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set device (GPU if available, otherwise CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Check CUDA availability
if torch.cuda.is_available():
    print("CUDA is available")
    print(f"Device count: {torch.cuda.device_count()}")
    print(f"Current device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available, using CPU")

Using device: cpu
CUDA is not available, using CPU


In [3]:
# Load the fine-tuned model and tokenizer
model_path = r"C:\Users\Fawaz\Desktop\AI_Text_Detector\human-ai-model"

# Load tokenizer
tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)
print(f"Tokenizer loaded from: {model_path}")

# Load model
model = DistilBertForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()
print(f"Model loaded from: {model_path}")
print(f"Model moved to device: {device}")

Tokenizer loaded from: C:\Users\Fawaz\Desktop\AI_Text_Detector\human-ai-model
Model loaded from: C:\Users\Fawaz\Desktop\AI_Text_Detector\human-ai-model
Model moved to device: cpu


In [4]:
# Define the prediction function
def predict_text_source(text: str, model, tokenizer):
    """
    Predict whether a given text is Human or AI-generated.
    
    Args:
        text (str): The text to classify
        model: The loaded DistilBERT model
        tokenizer: The loaded tokenizer
    
    Returns:
        str: Prediction result with confidence score
    """
    # Tokenize the input text
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        padding="max_length", 
        max_length=128
    ).to(device)
    
    # Make prediction
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1).squeeze()
        pred = torch.argmax(probs).item()
    
    # Convert prediction to label
    label = "Human" if pred == 0 else "AI"
    confidence = probs[pred].item()
    
    return f"{label} (Confidence: {confidence:.3f})"

In [5]:
# Test the model with some example texts
test_texts = [
    "This is a simple human-written text about everyday life.",
    "The artificial intelligence system generated this response using advanced language models.",
    "I went to the store yesterday and bought some groceries.",
    "Based on the provided context, I can analyze the data and provide insights.",
    "The weather is really nice today, I think I'll go for a walk."
]

print("Testing the model with example texts:")
print("=" * 60)

for i, text in enumerate(test_texts, 1):
    prediction = predict_text_source(text, model, tokenizer)
    print(f"\n{i}. Text: {text}")
    print(f"   Prediction: {prediction}")

Testing the model with example texts:

1. Text: This is a simple human-written text about everyday life.
   Prediction: AI (Confidence: 1.000)

2. Text: The artificial intelligence system generated this response using advanced language models.
   Prediction: AI (Confidence: 1.000)

3. Text: I went to the store yesterday and bought some groceries.
   Prediction: AI (Confidence: 0.998)

4. Text: Based on the provided context, I can analyze the data and provide insights.
   Prediction: AI (Confidence: 1.000)

5. Text: The weather is really nice today, I think I'll go for a walk.
   Prediction: AI (Confidence: 0.997)


In [ ]:
# Interactive prediction function
def interactive_prediction():
    """
    Interactive function to test the model with user input.
    """
    print("\n" + "="*60)
    print("INTERACTIVE PREDICTION MODE")
    print("="*60)
    print("Enter text to classify (or 'quit' to exit):")
    
    while True:
        user_input = input("\nEnter text: ")
        
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("Exiting...")
            break
        
        if user_input.strip():
            prediction = predict_text_source(user_input, model, tokenizer)
            print(f"Prediction: {prediction}")
        else:
            print("Please enter some text.")

# Uncomment the line below to run interactive mode
interactive_prediction()


INTERACTIVE PREDICTION MODE
Enter text to classify (or 'quit' to exit):


In [ ]:
# Function to get detailed prediction probabilities
def get_detailed_prediction(text: str, model, tokenizer):
    """
    Get detailed prediction probabilities for both Human and AI classes.
    
    Args:
        text (str): The text to classify
        model: The loaded DistilBERT model
        tokenizer: The loaded tokenizer
    
    Returns:
        dict: Detailed prediction results
    """
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        padding="max_length", 
        max_length=128
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        probs = softmax(outputs.logits, dim=1).squeeze()
        
    return {
        "human_probability": probs[0].item(),
        "ai_probability": probs[1].item(),
        "prediction": "Human" if probs[0] > probs[1] else "AI",
        "confidence": max(probs[0].item(), probs[1].item())
    }

# Test detailed prediction
example_text = "This is a test text to demonstrate the detailed prediction functionality."
detailed_result = get_detailed_prediction(example_text, model, tokenizer)

print("Detailed Prediction Results:")
print(f"Text: {example_text}")
print(f"Human Probability: {detailed_result['human_probability']:.3f}")
print(f"AI Probability: {detailed_result['ai_probability']:.3f}")
print(f"Prediction: {detailed_result['prediction']}")
print(f"Confidence: {detailed_result['confidence']:.3f}")